Use:
    Multiple sequence alignments
    Balanced training (33% proportion)
    Structure context training
    Jury of networks
        Majority vote of a set of 12 different networks

Cross-validated on 7 tests to get 3 state prediction of 69.7%

In [217]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense
import tensorflow as tf
from sklearn.metrics import matthews_corrcoef
from sklearn.utils.class_weight import compute_class_weight

# Function to parse data from a file
def parse_data(file_path):
    sequences, secondary_structures = [], []
    parsing_sequences = False
    with open(file_path, "r") as file:
        sequence = ""
        secondary_structure = ""
        for line in file:
            line = line.strip()
            if line == "<>" and not parsing_sequences:
                parsing_sequences = True
            elif line == "end" or line == "<end>":
                parsing_sequences = False
                sequences.append(sequence)
                secondary_structures.append(secondary_structure)
                sequence = ""
                secondary_structure = ""
            elif parsing_sequences and line == "<>":
                sequences.append(sequence)
                secondary_structures.append(secondary_structure)
                sequence = ""
                secondary_structure = ""
            elif parsing_sequences:
                parts = line.split()
                if len(parts) == 2:
                    amino_acid, sec_structure = parts
                    sequence += amino_acid
                    secondary_structure += sec_structure
    return sequences, secondary_structures

# Function to write sequences to a FASTA file
def write_fasta(sequences, output_file):
    with open(output_file, "w") as file:
        for i, sequence in enumerate(sequences):
            file.write(f">Sequence_{i+1}\n")
            file.write(f"{sequence}\n")

# Function to read aligned sequences from a file
def read_aligned_sequences(file_path):
    sequences = {}
    with open(file_path, "r") as file:
        sequence_number = None
        sequence = ""
        for line in file:
            line = line.strip()
            if line.startswith(">"):
                if sequence_number is not None:
                    sequences[sequence_number] = sequence
                sequence_number = int(line.split("_")[1])
                sequence = ""
            else:
                sequence += line
        if sequence_number is not None and sequence:
            sequences[sequence_number] = sequence
    return sequences

# Function to align structures with sequences
def align_structure(structures, sequences):
    aligned = {}
    for i in sequences.keys():
        align = []
        sequence = sequences[i]
        structure = structures[i-1]

        j = 0
        for char in sequence:
            if char == '-':
                align.append('-')
            else:
                align.append(structure[j])
                j += 1
        aligned[i] = align
    return aligned

# Function to encode a sequence using one-hot encoding
def encode_sequence(sequence):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    num_amino_acids = len(amino_acids)
    encoded_seq = np.zeros((len(sequence), num_amino_acids), dtype=int)
    for i, aa in enumerate(sequence):
        if aa in amino_acids:
            encoded_seq[i, amino_acids.index(aa)] = 1
    return encoded_seq

# Function to encode a structure
def encode_structure(structure):
    mapping = {'-': 4, '_': 0, 'e': 1, 'h': 2}
    return [mapping[char] for char in structure]

# Function to preprocess data for training
def preprocess_data(aligned_sequences, aligned_sequence_structures, window_size):
    X = []
    y = []

    for seq_id, sequence in aligned_sequences.items():
        encoded_seq = encode_sequence(sequence)
        encoded_struct = encode_structure(aligned_sequence_structures[seq_id])

        for i in range(len(sequence) - window_size + 1):
            X.append(encoded_seq[i:i+window_size])
            y.append(encoded_struct[i+window_size//2])

    # Remove instances with padding
    for i in range(len(y) - 1, -1, -1):
        if y[i] == 4:
            X.pop(i)
            y.pop(i)

    return np.array(X), np.array(y)

def calculate_accuracy(y_true, y_pred):

    if len(y_pred.shape) == 2:
        y_pred_labels = np.argmax(y_pred, axis=1)
    else:
        y_pred_labels = y_pred

    class_accuracies = {}
    for class_idx in range(3):
        class_pred_labels = y_pred_labels[y_true == class_idx]
        class_true_labels = y_true[y_true == class_idx]

        class_accuracy = np.sum(class_pred_labels == class_true_labels) / len(class_true_labels)

        class_names = {0: 'coil', 1: 'β-sheet', 2: 'α-helix'}
        class_accuracies[class_names[class_idx]] = class_accuracy

    for class_name, accuracy in class_accuracies.items():
        print(f"Accuracy for class {class_name}: {accuracy}")

    total_correct_predictions = np.sum(y_pred_labels == y_true)
    total_accuracy = total_correct_predictions / len(y_true)

    print("Total Accuracy:", total_accuracy)

In [218]:
window_size = 13
train_path = 'Q_and_s_data/protein-secondary-structure.train.txt'

# Preprocess training data
train_sequences, train_structures = parse_data(train_path)
aligned_sequences = read_aligned_sequences("training_msa.txt")
aligned_sequence_structures = align_structure(train_structures, aligned_sequences)
X_train, y_train = preprocess_data(aligned_sequences, aligned_sequence_structures, window_size)

# Compute class weights
class_weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(y_train), y = y_train)
class_weight_dict = dict(enumerate(class_weights))

In [219]:
network_1 = Sequential([
  Input(shape=(window_size, 20)),
  LSTM(units=273, activation='relu'),
  Dense(units=3, activation='softmax', name='output_1'),
])
# Update network compilation and training (using MSE)
network_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
network_1.fit(X_train, y_train, epochs=10, batch_size=32, class_weight=class_weight_dict)

Epoch 1/10


ValueError: Exception encountered when calling LSTMCell.call().

[1mDimensions must be equal, but are 20 and 21 for '{{node sequential_153_1/lstm_131_1/lstm_cell_1/MatMul}} = MatMul[T=DT_FLOAT, grad_a=false, grad_b=false, transpose_a=false, transpose_b=false](sequential_153_1/lstm_131_1/strided_slice_2, sequential_153_1/lstm_131_1/lstm_cell_1/Cast/ReadVariableOp)' with input shapes: [?,20], [21,1092].[0m

Arguments received by LSTMCell.call():
  • inputs=tf.Tensor(shape=(None, 20), dtype=float32)
  • states=('tf.Tensor(shape=(None, 273), dtype=float32)', 'tf.Tensor(shape=(None, 273), dtype=float32)')
  • training=True

In [215]:
test_path = 'Q_and_s_data/protein-secondary-structure.test.txt'

test_sequences, test_structures = parse_data(test_path)
test_aligned_sequences = read_aligned_sequences("test_msa.txt")
test_aligned_sequence_structures = align_structure(test_structures, test_aligned_sequences)
X_test, y_test = preprocess_data(test_aligned_sequences, test_aligned_sequence_structures, window_size)

# Evaluate the model
loss, accuracy = network_1.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.3205 - loss: 1.1889
Test Loss: 1.1679987907409668, Test Accuracy: 0.33304622769355774


In [216]:
y_pred_prob = network_1.predict(X_test)

calculate_accuracy(y_test, y_pred_prob)


109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step
Accuracy for class coil: 0.1601685985247629
Accuracy for class β-sheet: 0.543978349120433
Accuracy for class α-helix: 0.5366430260047281
Total Accuracy: 0.3330462245190927


In [127]:
num_networks = 12
networks = []

for _ in range(num_networks):
    model = Sequential([
        Input(shape=(window_size, 20)),
        LSTM(units=273),
        Dense(units=3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, class_weight=class_weight_dict)
    networks.append(model)

Epoch 1/10
565/565 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.4783 - loss: 0.9914
Epoch 2/10
565/565 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.5620 - loss: 0.9070
Epoch 3/10
565/565 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.5869 - loss: 0.8720
Epoch 4/10
565/565 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.5864 - loss: 0.8504
Epoch 5/10
565/565 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.6072 - loss: 0.8187
Epoch 6/10
565/565 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.6356 - loss: 0.7695
Epoch 7/10
565/565 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.6641 - loss: 0.7095
Epoch 8/10
565/565 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.6898 - loss: 0.6469
Epoch 9/10
565/565 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.7196 - loss: 0.5868
Epoch 10/10
565/565 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.7448 - loss: 0.5245
Epoch 1/10
565/565 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.4865 - loss: 0.9914
Epoch 2/10
565/565 ━━━━━━━━━━━━━━

In [128]:
# Aggregate predictions using majority vote
def majority_vote(predictions):
    aggregated_predictions = []
    for preds in predictions:
        aggregated_predictions.append(np.argmax(preds, axis=1))

    ensemble_predictions = np.array(aggregated_predictions)
    majority_vote_result = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=ensemble_predictions)
    
    return majority_vote_result

predictions = []
for model in networks:
    predictions.append(model.predict(X_test))

# Aggregate predictions using majority vote
ensemble_predictions = majority_vote(np.array(predictions))

109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step


In [129]:
from sklearn.model_selection import KFold

def majority_vote(predictions):
    aggregated_predictions = []
    for preds in predictions:
        aggregated_predictions.append(np.argmax(preds, axis=1))

    ensemble_predictions = np.array(aggregated_predictions)
    majority_vote_result = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=ensemble_predictions)
    
    return majority_vote_result

# Define the number of networks in the ensemble
num_networks = 12

# Function to create and train the jury of networks
def train_jury(X, y, kfold):
  networks = []
  fold_accuracies = []
  for train_index, test_index in kfold.split(X):
    X_train, X_val = X[train_index], X[test_index]
    y_train, y_val = y[train_index], y[test_index]

    # Create ensemble of networks
    ensemble_networks = []
    for _ in range(num_networks):
      model = Sequential([
          Input(shape=(window_size, 20)),
          LSTM(units=273),
          Dense(units=3, activation='softmax')
      ])
      model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
      model.fit(X_train, y_train, epochs=10, batch_size=32, class_weight=class_weight_dict)
      ensemble_networks.append(model)

    # Make predictions using majority vote on validation set
    predictions = []
    for model in ensemble_networks:
      predictions.append(model.predict(X_val))
    ensemble_predictions = majority_vote(np.array(predictions))

    # Evaluate ensemble on validation set and store accuracy
    _, accuracy = model.evaluate(X_val, ensemble_predictions)
    fold_accuracies.append(accuracy)

  # Print average accuracy across folds
  print(f"Average Accuracy Across Folds: {np.mean(fold_accuracies)}")
  return ensemble_networks

# Perform 7-fold cross-validation
kfold = KFold(n_splits=7, shuffle=True)
ensemble_networks = train_jury(X_train, y_train, kfold)


Epoch 1/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - accuracy: 0.4670 - loss: 1.0043
Epoch 2/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.5560 - loss: 0.9200
Epoch 3/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.5622 - loss: 0.8985
Epoch 4/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.5883 - loss: 0.8645
Epoch 5/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.5908 - loss: 0.8474
Epoch 6/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.6215 - loss: 0.8114
Epoch 7/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.6346 - loss: 0.7639
Epoch 8/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.6626 - loss: 0.7046
Epoch 9/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.6903 - loss: 0.6522
Epoch 10/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.7219 - loss: 0.5902
Epoch 1/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - accuracy: 0.4729 - loss: 1.0017
Epoch 2/10
485/485 ━━━━━━━━━━━━

In [130]:
# Evaluate the ensemble predictions
loss, accuracy = model.evaluate(X_test, ensemble_predictions)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


109/109 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8337 - loss: 0.4953
Test Loss: 0.5350601673126221, Test Accuracy: 0.8225667476654053


In [131]:
calculate_accuracy(y_test, ensemble_predictions)

Accuracy for class coil: 0.6396206533192834
Accuracy for class β-sheet: 0.46820027063599456
Accuracy for class α-helix: 0.4787234042553192
Total Accuracy: 0.5641688199827735
